# Zombie modelling


In [2]:
# Import dependencies


import os
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import xarray
import rioxarray

from geocube.api.core import make_geocube